**Aspect Based Sentiment Analysis:**

In this task we create a classification for restaurant reviews. The challenge faced in sentiment analysis is that, one can express positive as well as negative thoughts about an incidence at once. This leads to a better understanding of machines to judge whether the corrosponding opinion is positive or negative. Unlike previous sentiment analysis methods, eg: VADER, it is inefficient to generalize the review polarity based on aggregated understanding of words. 

In this task I have performed sentiment analysis based on aspects in reviews. 


The files are uploaded on shared drive for faster upload in colab. 

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.5MB/s 


In [0]:
glove_file = "https://drive.google.com/open?id=1QJxAC-USuELr7m7lXNfI33JvxLteJVIz"
xml_gold_data = "https://drive.google.com/open?id=1BAoAEooRgaTNRCpKrJEMVctzKZ8TzQjF"

In [0]:
fluff1, id1 = glove_file.split('=')
fluff2, id2 = xml_gold_data.split('=')

downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('foursquare_gold.xml')

downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('glove.6B.200d.txt')

**Import all the libraries and packages:**

In [0]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import spacy

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers import merge
from keras.utils import np_utils

from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gensim.models import KeyedVectors

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import load_model


**Data:** 

Primarily worked on foursquare_gold data. 

The XML file is parsed here and data is converted in pandas dataframe. 
Each review in XML tree has corrosponding sentences and text. I have defined this problem as a supervised machine learning problem. 

Training data consists of reviews with each aspect term and categories associated with it. For example, in case 4 we have 4 sub sentences. In our training we associate all four opinions with the same text for our classifier to learn from. 

Target (Y labels for training) are expressed as polarity. 

In [0]:
tree = ET.parse('foursquare_gold.xml')
root = tree.getroot()



In [0]:
root

<Element 'Reviews' at 0x7f5772923b88>

In [0]:
for child in root.iter():
    print(child.tag, child.attrib)

Reviews {}
Review {'rid': '0'}
sentences {}
sentence {'id': '0:0'}
text {}
Opinions {}
Opinion {'target': 'coffee', 'category': 'DRINKS#QUALITY', 'polarity': 'negative', 'from': '16', 'to': '22'}
Review {'rid': '1'}
sentences {}
sentence {'id': '1:0'}
text {}
Opinions {}
Opinion {'target': 'hype', 'category': 'AMBIENCE#GENERAL', 'polarity': 'negative', 'from': '22', 'to': '26'}
Opinion {'target': 'NULL', 'category': 'RESTAURANT#PRICES', 'polarity': 'negative', 'from': '0', 'to': '0'}
Review {'rid': '2'}
sentences {}
sentence {'id': '2:0'}
text {}
Opinions {}
Opinion {'target': 'carrot juice', 'category': 'DRINKS#QUALITY', 'polarity': 'positive', 'from': '16', 'to': '28'}
Review {'rid': '3'}
sentences {}
sentence {'id': '3:0'}
text {}
Opinions {}
Opinion {'target': 'NULL', 'category': 'RESTAURANT#GENERAL', 'polarity': 'positive', 'from': '0', 'to': '0'}
Review {'rid': '4'}
sentences {}
sentence {'id': '4:0'}
text {}
Opinions {}
Opinion {'target': 'NULL', 'category': 'RESTAURANT#GENERAL'

In [0]:
reviews_id = []
sentence_id = []
reviews = []
aspect_words = []
aspect_categories = []
aspect_polarities = []



for child in root.findall('Review'):
    review_id = child.get('rid')
    #print(review_id)
    
    
    for sent in child.findall('sentences'):
        sentences = sent.findall('sentence')
        for sents in sentences:
            sent_id = sents.get('id')
            if len(sents.findall('Opinions')) == 0:
                print(review_id)
                reviews_id.append(review_id)
                sentence_id.append(sent_id)
                #reviews_id.append(review_id)
                print(sent_id)
                review = sents.find('text').text
                reviews.append(review)
                print(review)
                aspect_words.append("NULL")
                aspect_categories.append("NULL")
                aspect_polarities.append("neutral")
                print("NULL","NULL","neutral")

            for sent1 in sents.findall('Opinions'):
                opinions = sent1.findall('Opinion')
                for opinion in opinions:

                    aspect_word = opinion.get('target')
                    aspect_category = opinion.get('category')
                    aspect_polarity = opinion.get('polarity')
                    review = sents.find('text').text
                    print(review_id)
                    reviews_id.append(review_id)
                    print(sent_id)
                    sentence_id.append(sent_id)
                    print(review)
                    reviews.append(review)
                    print(aspect_word, aspect_category, aspect_polarity)

                    aspect_words.append(aspect_word)
                    aspect_categories.append(aspect_category)
                    aspect_polarities.append(aspect_polarity)
        

0
0:0
2 words -filter coffee :-)
coffee DRINKS#QUALITY negative
1
1:0
Not worth the price & hype
hype AMBIENCE#GENERAL negative
1
1:0
Not worth the price & hype
NULL RESTAURANT#PRICES negative
2
2:0
You have to try carrot juice
carrot juice DRINKS#QUALITY positive
3
3:0
Im in love with it
NULL RESTAURANT#GENERAL positive
4
4:0
Total rip off!!
NULL RESTAURANT#GENERAL negative
4
4:1
They tried to change us more than double service charge on our bill.
NULL SERVICE#GENERAL negative
4
4:2
The small pizza is tiny and up the road you can get a 3 course meal for 15 soles!
small pizza FOOD#STYLE_OPTIONS negative
4
4:3
AVOID THIS PLACE!!!!
NULL RESTAURANT#GENERAL negative
5
5:0
The best bar in Hikkaduwa.
bar RESTAURANT#GENERAL positive
5
5:1
U can find here everything u want)
NULL RESTAURANT#GENERAL positive
6
6:0
Nice authentic tradition South Indian food..
food FOOD#QUALITY positive
6
6:1
Nice ambience and the lighting is good
ambience AMBIENCE#GENERAL positive
6
6:1
Nice ambience and the ligh

In [0]:
data = pd.DataFrame([reviews_id, sentence_id, reviews, aspect_words, aspect_categories, aspect_polarities])

In [0]:
data= data.T


In [0]:
data.head(6)

,0,1,2,3,4,5
0,0,0:0,2 words -filter coffee :-),coffee,DRINKS#QUALITY,negative
1,1,1:0,Not worth the price & hype,hype,AMBIENCE#GENERAL,negative
2,1,1:0,Not worth the price & hype,NULL,RESTAURANT#PRICES,negative
3,2,2:0,You have to try carrot juice,carrot juice,DRINKS#QUALITY,positive
4,3,3:0,Im in love with it,NULL,RESTAURANT#GENERAL,positive
5,4,4:0,Total rip off!!,NULL,RESTAURANT#GENERAL,negative


In [0]:
data = data.rename(index=str, columns={0: "review_id", 1: "sentence_id", 2:"reviews", 
                                       3:"aspect_words", 4:"aspect_categories",5:"aspect_polarities"})

In [0]:
df = data
df.head(5)

,review_id,sentence_id,reviews,aspect_words,aspect_categories,aspect_polarities
0,0,0:0,2 words -filter coffee :-),coffee,DRINKS#QUALITY,negative
1,1,1:0,Not worth the price & hype,hype,AMBIENCE#GENERAL,negative
2,1,1:0,Not worth the price & hype,NULL,RESTAURANT#PRICES,negative
3,2,2:0,You have to try carrot juice,carrot juice,DRINKS#QUALITY,positive
4,3,3:0,Im in love with it,NULL,RESTAURANT#GENERAL,positive


**Data Preprocessing:** 

1. Reviews
2. Aspect words
3. Aspect categories
4. Aspect polarities


We will discuss aspect categories first. For preprocessing, I first focused on aspect categories and aspect words. In the instructions given, NULL value for aspect words indicates that there is no aspect term available for the review. When inspected the data, I observed that there are certain samples where opinion is not available at all. An example can be seen as sentence 17, that has only text but no aspect terms, categories and polarity. I decided to go ahead with the categories as one hot representation for first pass. 

To take care of NULL values in aspect words, I checked if the noun in the text can be closer to expressing aspect of the sentence. Thus, I extracted nouns and appended these values back to aspect words to fill in the NULL values. There are certain sentences where we truly do not have aspect terms available and for those I have accepted NULL as aspect word and moved on. 

For those samples where the aspect words are derived, the next challenge was to identify aspect categories. I trained simple neural network to predict the corresponding aspect category for the review. After extracting the values for predicted aspect categories, I appended those in main dataframe. 

In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [0]:
label_encoder = LabelEncoder()
int_aspect_category = label_encoder.fit_transform(df.aspect_categories)
dummy_category = to_categorical(int_aspect_category)

In [0]:
max(int_aspect_category) # there are total 13 aspect categories: 0-11

12

In [0]:
# align all the text values together where opinion == null 
# then calculate the aspect term for these text entries 
# train the model to identify text categories and assign the value to categories 
# merge the clean data and at the end perform sentiment classification. 


In [0]:
# create aspect terms for missing text opinion values: 
import spacy
nlp = spacy.load('en')

In [0]:
df_null = df[df['aspect_words'] == 'NULL']

In [0]:
missed_aspect_terms = []


for review in nlp.pipe(df_null.reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    missed_aspect_terms.append(' '.join(chunks))


In [0]:
missed_aspect_terms

['price hype',
 'love',
 '',
 'charge bill',
 '',
 'everything',
 'Thumbs',
 '',
 '',
 'mocha chocolate',
 '',
 '',
 '',
 'Burgers pickles rings shake area',
 'cash',
 'Menu readers',
 '',
 'prices',
 'month',
 '',
 '',
 '',
 'coffee register back',
 'prices',
 'aesthetic property',
 'time',
 '',
 '',
 'tortillas tortillas',
 'water pitcher',
 '',
 'risotto',
 'coffee oatmeal way day',
 '',
 '',
 'cars days',
 'evening riverside',
 'place people food',
 'ave',
 'location',
 'place fish sea food prices',
 'place fish sea food prices',
 'meals salads snacks menu',
 'try bathing',
 'friends',
 'place',
 'lee starvin',
 '',
 'face',
 '',
 'reservatiom reservation',
 '',
 'price ambiance music',
 'dollars size baby',
 '',
 'Sundays',
 'Value money',
 'Value money',
 'experience waitress care tables',
 '',
 '',
 'jam',
 '',
 'mimosas wait',
 'chain',
 '',
 '',
 'fi',
 'photo',
 'taste variety beamwallet',
 'experience management issue',
 '',
 '',
 '',
 'issue',
 'fry naan',
 'hotpot town',
 

In [0]:
for i in range(len(missed_aspect_terms)):
    if missed_aspect_terms[i] == '':
        missed_aspect_terms[i] = 'NULL'

In [0]:
missed_aspect_terms

['price hype',
 'love',
 'NULL',
 'charge bill',
 'NULL',
 'everything',
 'Thumbs',
 'NULL',
 'NULL',
 'mocha chocolate',
 'NULL',
 'NULL',
 'NULL',
 'Burgers pickles rings shake area',
 'cash',
 'Menu readers',
 'NULL',
 'prices',
 'month',
 'NULL',
 'NULL',
 'NULL',
 'coffee register back',
 'prices',
 'aesthetic property',
 'time',
 'NULL',
 'NULL',
 'tortillas tortillas',
 'water pitcher',
 'NULL',
 'risotto',
 'coffee oatmeal way day',
 'NULL',
 'NULL',
 'cars days',
 'evening riverside',
 'place people food',
 'ave',
 'location',
 'place fish sea food prices',
 'place fish sea food prices',
 'meals salads snacks menu',
 'try bathing',
 'friends',
 'place',
 'lee starvin',
 'NULL',
 'face',
 'NULL',
 'reservatiom reservation',
 'NULL',
 'price ambiance music',
 'dollars size baby',
 'NULL',
 'Sundays',
 'Value money',
 'Value money',
 'experience waitress care tables',
 'NULL',
 'NULL',
 'jam',
 'NULL',
 'mimosas wait',
 'chain',
 'NULL',
 'NULL',
 'fi',
 'photo',
 'taste variety 

In [0]:
j = 0
for i in range(len(df)):
    if df.iloc[i,3] == 'NULL':
        df.iloc[i,3] = missed_aspect_terms[j]
        j=j+1

In [0]:
df.head(10)

,review_id,sentence_id,reviews,aspect_words,aspect_categories,aspect_polarities
0,0,0:0,2 words -filter coffee :-),coffee,DRINKS#QUALITY,negative
1,1,1:0,Not worth the price & hype,hype,AMBIENCE#GENERAL,negative
2,1,1:0,Not worth the price & hype,price hype,RESTAURANT#PRICES,negative
3,2,2:0,You have to try carrot juice,carrot juice,DRINKS#QUALITY,positive
4,3,3:0,Im in love with it,love,RESTAURANT#GENERAL,positive
5,4,4:0,Total rip off!!,NULL,RESTAURANT#GENERAL,negative
6,4,4:1,They tried to change us more than double servi...,charge bill,SERVICE#GENERAL,negative
7,4,4:2,The small pizza is tiny and up the road you ca...,small pizza,FOOD#STYLE_OPTIONS,negative
8,4,4:3,AVOID THIS PLACE!!!!,NULL,RESTAURANT#GENERAL,negative
9,5,5:0,The best bar in Hikkaduwa.,bar,RESTAURANT#GENERAL,positive


In [0]:
df_category_null = df[df['aspect_categories']== 'NULL']

In [0]:
df_cat_notnull = df[df['aspect_categories'] != 'NULL']

In [0]:
#df_cat_notnull

In [0]:
# preprocess the reviews: 

# 1. Remove stop words : a, an, the, it, in,  ..
# 2. Remove punctuations: .,_-?|""
# 3. Try to see if emoji can be decoded : :), :-), )
# 4. stemming
# 5. lemma

**Preprocessing of reviews:** 

After taking care of aspect words and categories, the main part was reviews! Here I would like to make a note, most of the reviews that had missing polarity were simple neutral sentences. Hence I have defined polarities for such sentences to be 'neutral'.

For reviews, following is the pipeline: 

**1. Stopwords:** 

Using NLTK, stopwords could be easily removed but here that became challenging. I first tried a simple list comprehension with in-built stop words to get rid of stopwords from reviews but this changed the text drastically. In text mining, when the text has smaller size or length, getting rid of the stopwords can be detrimental while understanding the overall meaning of stop words. 

To tackle this, I simply created a list of some of the most obvious stopwords that should not alter the meaning of our reviews. Using list comprehension I removed only these words from all reviews. 

**2. Punctuation:** 

Using regular expressions, I removed all punctuations. Here I would like to menton that if I had more time, I am motivated to understand how having one vs three exclamations add weight to the expression. For example wow! and wow!!! could be same or may not make a difference. 

Also I want to spend time understanding emojis. Here not many text had emojis like :-), :) but it would be interesting to see if decoding these can help us any better. Need to search for vectors/embeddings for such symbols to ensure that classifier will definitely get some more information. 

For now, I have removed all punctuations but this thought remains on my mind for learning. 

**3. Lower case:** 

To ensure that 'GREAT' and 'great' are same words and the vectors are equal, the text has been turned to lower case. 

**4. Stemming and Lemmatization:** 

In this task, to get rid of 'ing', 's', 'es' type of text, I tried Lancaster stemmer (works better than porter stemmer) however the modifications were drastic. To ensure that with such small sequence, we retain maximum information, I opted for lemmatization. 

For retaining the roots of every word, I used WordNetLemmatizer. Here I have used roots of nouns and verbs primarily. By joining all the preprocessed tokens back to sentence format, appended the list and saved it in a variable. 

In the dataframe, this preprocessed text is appended for further training. 




In [0]:
df_reviews = df['reviews']

In [0]:
df_reviews = str(df_reviews)

In [0]:
import re 
import nltk
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 


In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
ls = LancasterStemmer()
lemma = WordNetLemmatizer()

In [0]:
stop_words = ['a', 'an', 'the', 'at','it', 'of', 'for', 'in', 'this', 'that', 'if', 'as', 'to', 're', 'there', 'here']

In [0]:
preprocessed_reviews = []

for i in range(0, len(df['reviews'])):
    review = re.sub('[^a-zA-Z]', ' ', df['reviews'][i])
    review = re.sub("[^\w]",' ', df['reviews'][i])
    review = review.lower()
    review = review.split()
    review = [word for word in review if word not in stop_words]
    review = [lemma.lemmatize(j) for j in review] 
    review = [lemma.lemmatize(j,'v') for j in review]
    #review = [ls.stem(word) for word in review]
    review = ' '.join(review)
    preprocessed_reviews.append(review)

In [0]:
preprocessed_reviews[1]

'not worth price hype'

In [0]:
df.reviews[1]

'Not worth the price & hype'

In [0]:
for i in range(len(df)):
  df.reviews[i] = preprocessed_reviews[i]

In [0]:
df.reviews[1]

'not worth price hype'

In [0]:
t = Tokenizer()


In [0]:
t.fit_on_texts(df_cat_notnull.reviews)
vocab_size = len(t.word_index) + 1
vocab_size

1887

In [0]:
aspect_tokenize_word = t.texts_to_matrix(df_cat_notnull.aspect_words)

In [0]:
label_encoder = LabelEncoder()
int_aspect_category = label_encoder.fit_transform(df_cat_notnull.aspect_categories)
dummy_category = to_categorical(int_aspect_category)

In [0]:
max(int_aspect_category) # there are total 12 aspect categories: 0-11

11

In [0]:
# create the aspect categories for missing values of aspect categories: 

In [0]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,), activation='relu'))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.fit(aspect_tokenize_word, dummy_category, epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1155/1155 [==============================] - 1s 1ms/step - loss: 2.2455 - acc: 0.4165
Epoch 2/50
1155/1155 [==============================] - 0s 120us/step - loss: 1.6512 - acc: 0.4580
Epoch 3/50
1155/1155 [==============================] - 0s 122us/step - loss: 1.3450 - acc: 0.5524
Epoch 4/50
1155/1155 [==============================] - 0s 123us/step - loss: 1.1060 - acc: 0.6468
Epoch 5/50
1155/1155 [==============================] - 0s 120us/step - loss: 0.9039 - acc: 0.7628
Epoch 6/50
1155/1155 [==============================] - 0s 120us/step - loss: 0.7428 - acc: 0.8078
Epoch 7/50
1155/1155 [==============================] - 0s 126us/step - loss: 0.6326 - acc: 0.8277
Epoch 8/50
1155/1155 [==============================] - 0s 120us/step - loss: 0.5497 - acc: 0.8390
Epoch 9/50
1155/1155 [==============================] - 0s 119us/step - loss: 0.4909 - acc: 0.8468
Epoch 10/50
1155/1155 [==============================] - 0s 119

In [0]:
df_category_null.reviews

25      They use BonSoy and if you get a mocha they us...
28                                         Ask Christian.
30                                             Really?!?!
35                     Just remember it's "cash only"! :)
39                                  Menu is on e readers.
61      If you order coffee at the front register, you...
68           Doesn't match the aesthetic of the property.
86                       Puts Moe's and Chipotle to shame
93                                Kind of turkish risotto
104                                 Plan to wait a while.
133          Farted In one of these cars, stank for days.
134     You can also get one of these in the evening a...
139                       Go to rockys on 2nd ave instead
140           Don't be fooled by the strip mall location.
145           Other meals, salads and snacks in the menu.
147     Stoked I was finally able to stop by after hea...
148           This place stands up with the best of them.
150           

In [0]:
df_category_null.head(10)

,review_id,sentence_id,reviews,aspect_words,aspect_categories,aspect_polarities
25,11,11:1,They use BonSoy and if you get a mocha they us...,mocha chocolate,NULL,neutral
28,12,12:1,Ask Christian.,NULL,NULL,neutral
30,13,13:1,Really?!?!,NULL,NULL,neutral
35,14,14:1,"Just remember it's ""cash only""! :)",cash,NULL,neutral
39,17,17:0,Menu is on e readers.,Menu readers,NULL,neutral
61,28,28:1,"If you order coffee at the front register, you...",coffee register back,NULL,neutral
68,30,30:3,Doesn't match the aesthetic of the property.,aesthetic property,NULL,neutral
86,39,39:0,Puts Moe's and Chipotle to shame,NULL,NULL,neutral
93,41,41:2,Kind of turkish risotto,risotto,NULL,neutral
104,47,47:0,Plan to wait a while.,NULL,NULL,neutral


In [0]:
new_review_aspect_tokenized = t.texts_to_matrix(df_category_null.aspect_words)

In [0]:
#new_review_aspect_tokenized

In [0]:
aspect_predictions = model.predict_classes(new_review_aspect_tokenized) 

In [0]:
aspect_categories_pred = label_encoder.inverse_transform(aspect_predictions)

In [0]:
aspect_categories_pred

array(['DRINKS#QUALITY', 'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL',
       'RESTAURANT#GENERAL', 'FOOD#STYLE_OPTIONS', 'DRINKS#QUALITY',
       'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL', 'FOOD#QUALITY',
       'RESTAURANT#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL',
       'RESTAURANT#GENERAL', 'LOCATION#GENERAL', 'FOOD#STYLE_OPTIONS',
       'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#GENERAL',
       'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL',
       'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL', 'FOOD#QUALITY',
       'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL',
       'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL', 'FOOD#QUALITY',
       'FOOD#QUALITY', 'FOOD#QUALITY', 'RESTAURANT#GENERAL',
       'RESTAURANT#GENERAL', 'FOOD#STYLE_OPTIONS', 'SERVICE#GENERAL',
       'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL',
       'LOCATION#GENERAL', 'FOOD#QUALITY', 'FOOD#QUALITY',
       'RESTAURANT#GENERAL', 'RESTAURANT#GENERAL',
       'REST

In [0]:
# map this back 

In [0]:
j=0
for i in range(len(df)):
    if df.aspect_categories[i] == "NULL":
        df.aspect_categories[i] = aspect_categories_pred[j]
        j = j+1

In [0]:
df.head(5)

,review_id,sentence_id,reviews,aspect_words,aspect_categories,aspect_polarities
0,0,0:0,2 word filter coffee,coffee,DRINKS#QUALITY,negative
1,1,1:0,not worth price hype,hype,AMBIENCE#GENERAL,negative
2,1,1:0,not worth price hype,price hype,RESTAURANT#PRICES,negative
3,2,2:0,you have try carrot juice,carrot juice,DRINKS#QUALITY,positive
4,3,3:0,im love with,love,RESTAURANT#GENERAL,positive


In [0]:
len(df)

1312

In [0]:
df['aspect_categories'] = df['aspect_categories'].str.replace('#',' ')

In [0]:
df.head(3)

,review_id,sentence_id,reviews,aspect_words,aspect_categories,aspect_polarities
0,0,0:0,2 word filter coffee,coffee,DRINKS QUALITY,negative
1,1,1:0,not worth price hype,hype,AMBIENCE GENERAL,negative
2,1,1:0,not worth price hype,price hype,RESTAURANT PRICES,negative


In [0]:
# prepare x and y 

In [0]:
x = df.iloc[:,2:5]
x.head(3)

,reviews,aspect_words,aspect_categories
0,2 word filter coffee,coffee,DRINKS QUALITY
1,not worth price hype,hype,AMBIENCE GENERAL
2,not worth price hype,price hype,RESTAURANT PRICES


In [0]:
label = {'positive':0,'negative':1,'neutral':2}
y = [label[i] for i in df['aspect_polarities']]
y = np_utils.to_categorical(y, 3)

In [0]:
len(df[df['aspect_polarities'] == 'positive'])

946

In [0]:
len(df[df['aspect_polarities'] == 'negative'])

191

In [0]:
len(df[df['aspect_polarities'] == 'neutral'])

175

In [0]:
# split train data:

**Training:** 

Considering we have less number of samples, I chose only 0.15 to be a test size. To ensure that every time the sample setting is same, I set random_state while splitting the data into train test. While re-running my code with different setting, this parameter ensures that under the exact same settings we are testing the performance of our model. 

**Embeddings:** 

Here I chose text to be simple converted into on-hot vectors followed by GloVe embeddings in the initial layer of neural network. The choice of GloVe over word2vec is simply intuitive. For this experiment, choosing 200d was sufficient for embeddings. 


**LSTM:** 

To ensure that this is not merely sentiment analysis classification task, I wanted to emphasize on aspect words as well as aspect categories. 

If we had a review that consists of 3-5 lines, talking about multiple factors together then I would have segregated an entire review into sub-reviews based on aspect terms. However, in this data, the text is short and have been represented wrt to aspect terms and categories in dataframe (for each opinion). This eliminates the need for re-forming review driven by aspect term. 

Considering the aspect term is equally important for network to classify the review, I used 3 LSTM and embeddings. First, I created embedding vector of reviews. Second LSTM contains embedding vectors of aspect words and third LSTM contains embedding vectors of aspect categories. 
These three LSTMs are then concatenated together to move on towards classification. 


By including vectors of aspect terms and categories separately with reviews, I wanted to ensure that the network learns precisely about all three factors while classifying the review. Traditionally, in theory we know that neural nets are good at predicting the patterns in data and one could naively say that it would have figured it out by merging the input too however in practice this works better when given concatenated inputs. 

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 101)

In [0]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1115, 3) (197, 3) (1115, 3) (197, 3)


In [0]:
# use neural nets for detecting sentiment: 

In [0]:
tokenize = Tokenizer()

In [0]:
tokenize.fit_on_texts(X_train["reviews"])
tokenize.fit_on_texts(X_train["aspect_words"])
tokenize.fit_on_texts(X_train["aspect_categories"])

tokenize.fit_on_texts(X_test["reviews"])
tokenize.fit_on_texts(X_test["aspect_words"])
tokenize.fit_on_texts(X_test["aspect_categories"])

In [0]:
review_train = tokenize.texts_to_sequences(X_train["reviews"])
aspect_words_train = tokenize.texts_to_sequences(X_train["aspect_words"])
aspect_categories_train = tokenize.texts_to_sequences(X_train["aspect_categories"])

In [0]:
review_test = tokenize.texts_to_sequences(X_test["reviews"])
aspect_words_test = tokenize.texts_to_sequences(X_test["aspect_words"])
aspect_categories_test = tokenize.texts_to_sequences(X_test["aspect_categories"])

In [0]:
maxlen = 80

In [0]:
train_1 = sequence.pad_sequences(review_train, maxlen=maxlen)
train_2 = sequence.pad_sequences(aspect_words_train, maxlen=maxlen)
train_3 = sequence.pad_sequences(aspect_categories_train, maxlen=maxlen)

In [0]:
test_1 = sequence.pad_sequences(review_test, maxlen=maxlen)
test_2 = sequence.pad_sequences(aspect_words_test, maxlen=maxlen)
test_3 = sequence.pad_sequences(aspect_categories_test, maxlen=maxlen)

In [0]:
# =====preapare embedding matrix=====
word_index = tokenize.word_index
num_words = len(word_index)

In [0]:


embeddings_index = {}
with open("glove.6B.200d.txt", encoding = 'utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs


embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
num_words

2151

In [0]:
train_data = [train_1, train_2, train_3]

In [0]:

batch_size = 512
nb_epochs = 50
lstm_dim = 200
embedding_dim = 200
max_features = 1000

In [0]:
from keras.layers.merge import concatenate
from keras.layers import Embedding, Input
from keras.models import Model
from keras.layers.core import Dense

In [0]:
# First LSTM
input_1 = Input(shape=(maxlen,), dtype='int32')
embedding_1 = Embedding(input_dim=len(embedding_matrix), output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, mask_zero=True, trainable=True)(input_1)
LSTM_1 = LSTM(embedding_dim, batch_input_shape=(batch_size, maxlen, embedding_dim), input_shape=(maxlen, embedding_dim))(embedding_1)

# Second LSTM
input_2 = Input(shape=(maxlen,), dtype='int32')
embedding_2 = Embedding(input_dim=len(embedding_matrix), output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, mask_zero=True, trainable=True)(input_2)
LSTM_2 = LSTM(embedding_dim, batch_input_shape=(batch_size, maxlen, embedding_dim), input_shape=(maxlen, embedding_dim))(embedding_2)

#Third lstm
input_3 = Input(shape=(maxlen,), dtype='int32')
embedding_3 = Embedding(input_dim=len(embedding_matrix), output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, mask_zero=True, trainable=True)(input_3)
LSTM_3 = LSTM(embedding_dim, batch_input_shape=(batch_size, maxlen, embedding_dim), input_shape=(maxlen, embedding_dim))(embedding_3)


# Merge
merged = concatenate([LSTM_1, LSTM_2, LSTM_3])


# Dense
dense_out = Dense(3, activation='softmax')(merged)

# build and compile model
model = Model(inputs=[input_1, input_2, input_3], outputs=[dense_out])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 80)           0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           (None, 80)           0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           (None, 80)           0                                            
__________________________________________________________________________________________________
embedding_37 (Embedding)        (None, 80, 200)      430400      input_37[0][0]                   
__________________________________________________________________________________________________
embedding_

In [0]:
model.fit([train_1, train_2, train_3],y_train,
         epochs = nb_epochs,
         batch_size = batch_size,
         validation_data = ([test_1,test_2,test_3], y_test)
         #callbacks = [EarlyStopping(monitor = 'val_loss', patience = 10)] 
         )

Train on 1115 samples, validate on 197 samples
Epoch 1/50
1115/1115 [==============================] - 10s 9ms/step - loss: 1.0060 - acc: 0.4744 - val_loss: 0.7032 - val_acc: 0.7411
Epoch 2/50
1115/1115 [==============================] - 2s 1ms/step - loss: 0.7334 - acc: 0.7175 - val_loss: 0.7126 - val_acc: 0.7411
Epoch 3/50
1115/1115 [==============================] - 2s 1ms/step - loss: 0.7147 - acc: 0.7184 - val_loss: 0.6999 - val_acc: 0.7513
Epoch 4/50
1115/1115 [==============================] - 2s 1ms/step - loss: 0.6661 - acc: 0.7435 - val_loss: 0.6946 - val_acc: 0.7614
Epoch 5/50
1115/1115 [==============================] - 2s 1ms/step - loss: 0.6368 - acc: 0.7785 - val_loss: 0.6811 - val_acc: 0.7563
Epoch 6/50
1115/1115 [==============================] - 2s 1ms/step - loss: 0.6109 - acc: 0.7821 - val_loss: 0.6511 - val_acc: 0.7665
Epoch 7/50
1115/1115 [==============================] - 2s 1ms/step - loss: 0.5760 - acc: 0.7901 - val_loss: 0.6234 - val_acc: 0.7817
Epoch 8/50
111

**Hyperparameter tunning and Architecture :** 


1. Batch size: 

Checked the batch size 32, 128, 512 and 512 worked the best. With 32 batchsize the network was badly overfitting. 

2. epochs: 

Tried 25 o 50 and for the nature of tis problem, the network learns better with 50 epochs. 

3. Hidden layers: 

Tried adding 1 and then 2 idden layers with relu activation to improve the performance of the model but it was not helping. 
Tried dropout 0.3 as well yet it did not help. 

( I deleted this code to avoid confusion)

4. To eliminate overfitting: 

Tried using early stopping and drop out with hidden layers but this did not help. 

The model shown above worked the best in all combinations tried so far. 


5. Hardware: 

Used 1 GPU here and that helped in speeding up the training. This network gets trained in no less than 5 to 7 min. Using CPU, LSTM converged slow. 

6. Cost function: categorical_crossentropy as we have 3 classes, 'positive', 'negative' and 'neutral'. 

7. Optimizer: Adam.







In [0]:
y_pred = model.predict([test_1,test_2,test_3], batch_size=batch_size)
score, acc = model.evaluate([test_1,test_2,test_3], y_test, batch_size=batch_size)

197/197 [==============================] - 0s 770us/step


In [0]:
print("Test score:", score)
print("Test accuracy:", acc)

Test score: 0.6708827018737793
Test accuracy: 0.817258894443512


**Output:** 

In [0]:
y_pred

array([[9.99755681e-01, 2.44378403e-04, 2.22322161e-08],
       [4.02230680e-01, 8.17004777e-03, 5.89599311e-01],
       [3.67037565e-01, 6.32701576e-01, 2.60806119e-04],
       [9.99019384e-01, 5.79210988e-04, 4.01340483e-04],
       [9.17905271e-01, 7.92727545e-02, 2.82185315e-03],
       [9.42146420e-01, 3.30123901e-02, 2.48411950e-02],
       [9.99870658e-01, 9.64231003e-05, 3.29257891e-05],
       [4.22200188e-04, 3.87659490e-01, 6.11918271e-01],
       [5.55787329e-03, 9.92384017e-01, 2.05807132e-03],
       [9.86893177e-01, 1.76737714e-03, 1.13393934e-02],
       [8.13769083e-03, 7.66518860e-05, 9.91785586e-01],
       [9.99860883e-01, 7.28929372e-05, 6.61006270e-05],
       [9.86208081e-01, 1.08102011e-03, 1.27109112e-02],
       [9.99998569e-01, 5.16135856e-07, 9.41225608e-07],
       [9.99999046e-01, 2.86304783e-07, 6.67256757e-07],
       [9.97805417e-01, 1.04249802e-05, 2.18406366e-03],
       [6.58403933e-01, 3.37307870e-01, 4.28816397e-03],
       [1.93157606e-03, 8.51664

In [0]:
y_pred_classes = y_pred.argmax(axis=-1)

**Class prediction**

In [0]:
y_pred_classes

array([0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 2, 2, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
actual_label = y_test.argmax(axis=-1)

In [0]:
actual_label

array([0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, 0, 2, 2, 0, 0, 1, 1, 0, 2, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])